## 기본 예시: 프롬프트 + 모델 + 출력 파서

가장 기본적이고 일반적인 사용 사례는 prompt 템플릿과 모델을 함께 연결하는 것입니다. 이것이 어떻게 작동하는지 보기 위해, 각 나라별 수도를 물어보는 Chain을 생성해 보겠습니다.


In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH01-Basic")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH01-Basic


## 프롬프트 템플릿의 활용

`PromptTemplate`

- 사용자의 입력 변수를 사용하여 완전한 프롬프트 문자열을 만드는 데 사용되는 템플릿입니다
- 사용법
  - `template`: 템플릿 문자열입니다. 이 문자열 내에서 중괄호 `{}`는 변수를 나타냅니다.
  - `input_variables`: 중괄호 안에 들어갈 변수의 이름을 리스트로 정의합니다.

`input_variables`

- input_variables는 PromptTemplate에서 사용되는 변수의 이름을 정의하는 리스트입니다.

In [3]:
from langchain_teddynote.messages import stream_response  # 스트리밍 출력
from langchain_core.prompts import PromptTemplate

`from_template()` 메소드를 사용하여 PromptTemplate 객체 생성


In [4]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], template='{country}의 수도는 어디인가요?')

In [5]:
# prompt 생성
prompt = prompt_template.format(country="대한민국")
prompt

'대한민국의 수도는 어디인가요?'

In [6]:
# prompt 생성
prompt = prompt_template.format(country="미국")
prompt

'미국의 수도는 어디인가요?'

In [7]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-3.5-turbo",
    max_tokens=2048,
    temperature=0.1,
)

## Chain 생성

### LCEL(LangChain Expression Language)

![lcel.png](./images/lcel.png)

여기서 우리는 LCEL을 사용하여 다양한 구성 요소를 단일 체인으로 결합합니다

```
chain = prompt | model | output_parser
```

`|` 기호는 [unix 파이프 연산자](<https://en.wikipedia.org/wiki/Pipeline_(Unix)>)와 유사하며, 서로 다른 구성 요소를 연결하고 한 구성 요소의 출력을 다음 구성 요소의 입력으로 전달합니다.

이 체인에서 사용자 입력은 프롬프트 템플릿으로 전달되고, 그런 다음 프롬프트 템플릿 출력은 모델로 전달됩니다. 각 구성 요소를 개별적으로 살펴보면 무슨 일이 일어나고 있는지 이해할 수 있습니다.


In [8]:
# prompt 를 PromptTemplate 객체로 생성합니다.
prompt = PromptTemplate.from_template("{topic} 에 대해 쉽게 설명해주세요.")

model = ChatOpenAI()

chain = prompt | model

### invoke() 호출

- python 딕셔너리 형태로 입력값을 전달합니다.(키: 값)
- invoke() 함수 호출 시, 입력값을 전달합니다.

In [9]:
# input 딕셔너리에 주제를 '인공지능 모델의 학습 원리'으로 설정합니다.
input = {"topic": "인공지능 모델의 학습 원리"}

In [10]:
# prompt 객체와 model 객체를 파이프(|) 연산자로 연결하고 invoke 메서드를 사용하여 input을 전달합니다.
# 이를 통해 AI 모델이 생성한 메시지를 반환합니다.
chain.invoke(input)

AIMessage(content='인공지능 모델의 학습 원리는 데이터를 입력으로 받아들이고, 이 데이터를 분석하여 패턴이나 규칙을 학습하는 과정입니다. \n\n가장 일반적인 학습 방법은 지도 학습이며, 이때는 입력 데이터와 정답 데이터가 함께 제공되어 모델이 이 두 데이터 사이의 관계를 학습합니다. 이를 통해 모델은 입력 데이터를 분석하고 예측을 수행할 수 있게 됩니다.\n\n또한, 비지도 학습은 입력 데이터만을 이용하여 모델이 스스로 데이터의 구조나 패턴을 발견하고 학습하는 방법입니다. 이를 통해 모델은 데이터 간의 관계를 파악하거나 군집화를 수행할 수 있습니다.\n\n마지막으로, 강화 학습은 환경과 상호작용하면서 보상을 최대화하는 방향으로 모델이 학습하는 방법입니다. 모델은 행동을 취하고 그 결과에 따라 보상을 받아 보상을 최대화하는 전략을 학습하게 됩니다.\n\n이러한 학습 방법을 통해 인공지능 모델은 데이터를 분석하고 패턴을 학습하여 예측이나 결정을 수행할 수 있게 됩니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 386, 'prompt_tokens': 33, 'total_tokens': 419, 'completion_tokens_details': {'reasoning_tokens': 0}, 'prompt_tokens_details': {'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-bcd9b6d9-3cf5-4d48-950e-2fb8aafea019-0', usage_metadata={'input_tokens': 33, 'output_tokens': 386, 'total_tokens': 419})

아래는 스트리밍을 출력하는 예시 입니다.

In [11]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

인공지능 모델의 학습 원리는 데이터를 기반으로 하는 것입니다. 모델은 입력된 데이터를 분석하고 패턴을 학습하여 문제를 해결하는 방법을 익히는 것입니다. 이러한 과정을 통해 모델은 예측을 하거나 판단을 내리는 능력을 향상시킬 수 있습니다. 학습 과정은 데이터를 입력받아 가중치를 조정하고 오차를 최소화하는 방향으로 학습을 진행합니다. 이렇게 학습된 모델은 새로운 데이터에 대해 정확한 예측을 할 수 있습니다.

### 출력파서(Output Parser)


In [12]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

Chain 에 출력파서를 추가합니다.

In [13]:
# 프롬프트, 모델, 출력 파서를 연결하여 처리 체인을 구성합니다.
chain = prompt | model | output_parser

In [14]:
# chain 객체의 invoke 메서드를 사용하여 input을 전달합니다.
input = {"topic": "인공지능 모델의 학습 원리"}
chain.invoke(input)

'인공지능 모델의 학습 원리는 데이터를 입력으로 받아 내부적으로 패턴을 학습하고 결과를 예측하는 것입니다. 이 과정은 크게 입력층, 은닉층, 출력층으로 구성된 신경망을 사용하여 이루어집니다. \n\n먼저, 모델은 입력층에서 데이터를 받아들이고 은닉층에서 이 데이터를 처리하여 패턴을 학습합니다. 이때, 학습을 통해 가중치와 편향을 조절하여 입력 데이터와 실제 결과 사이의 상관관계를 최대화하는 방향으로 모델을 학습시킵니다.\n\n학습이 완료되면, 모델은 새로운 데이터를 입력으로 받아 출력층에서 결과를 예측합니다. 이때, 모델이 학습한 패턴을 바탕으로 입력 데이터와 결과 간의 관계를 예측하게 됩니다.\n\n이렇게 인공지능 모델은 학습을 통해 데이터 간의 패턴을 파악하고 새로운 데이터에 대한 예측을 수행하는 것이 핵심 원리입니다.'

In [15]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

인공지능 모델의 학습 원리는 데이터를 입력으로 받아서 패턴을 학습하고, 이를 바탕으로 새로운 데이터를 예측하는 과정입니다. 

가장 일반적인 학습 방법은 지도학습으로, 이때 모델은 입력 데이터와 정답 레이블이 함께 제공됩니다. 모델은 입력 데이터와 정답 레이블 간의 관계를 학습하여 새로운 데이터에 대해 정확한 예측을 할 수 있게 됩니다.

학습 과정은 일반적으로 손실 함수를 사용하여 모델의 예측 값과 정답 레이블 간의 차이를 계산하고, 이를 최소화하기 위해 모델의 가중치를 조정하는 방식으로 이루어집니다. 이러한 반복적인 과정을 통해 모델은 점차적으로 더 나은 예측을 할 수 있도록 개선됩니다.

이러한 학습 원리를 통해 인공지능 모델은 데이터를 기반으로 패턴을 학습하고, 이를 활용하여 다양한 문제를 해결할 수 있습니다.

### 템플릿을 변경하여 적용

- 아래의 프롬프트 내용을 얼마든지 **변경** 하여 테스트 해볼 수 있습니다.
- `model_name` 역시 변경하여 테스트가 가능합니다.

In [16]:
template = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 주어진 상황에 맞는 영어 회화를 작성해 주세요.
양식은 [FORMAT]을 참고하여 작성해 주세요.

#상황:
{question}

#FORMAT:
- 영어 회화:
- 한글 해석:
"""

# 프롬프트 템플릿을 이용하여 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(template)

# ChatOpenAI 챗모델을 초기화합니다.
model = ChatOpenAI(model_name="gpt-4-turbo")

# 문자열 출력 파서를 초기화합니다.
output_parser = StrOutputParser()

In [17]:
# 체인을 구성합니다.
chain = prompt | model | output_parser

In [18]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
print(chain.invoke({"question": "저는 식당에 가서 음식을 주문하고 싶어요"}))

- 영어 회화:
  Customer: Hi, could I see the menu, please?
  Waiter: Of course! Here you go.
  Customer: Thank you. I’d like to order the grilled salmon with a side of asparagus.
  Waiter: Great choice! Would you like anything to drink?
  Customer: Could I have a glass of white wine, please?
  Waiter: Absolutely, I'll bring that right out for you.

- 한글 해석:
  손님: 안녕하세요, 메뉴판 좀 볼 수 있을까요?
  웨이터: 물론이죠! 여기 있습니다.
  손님: 감사합니다. 연어 구이와 아스파라거스 사이드를 주문하겠습니다.
  웨이터: 좋은 선택이네요! 음료는 무엇으로 드릴까요?
  손님: 화이트 와인 한 잔 주시겠어요?
  웨이터: 네, 바로 가져다 드리겠습니다.


In [19]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "저는 식당에 가서 음식을 주문하고 싶어요"})
# 스트리밍 출력
stream_response(answer)

#영어 회화:
- Excuse me, could I see the menu, please?
- I'd like to order the grilled salmon with a side of roasted vegetables.
- Could I have a glass of white wine with that?
- Thank you, that will be all for now.

#한글 해석:
- 실례합니다, 메뉴판 좀 볼 수 있을까요?
- 구운 연어와 구운 채소 사이드를 주문하고 싶어요.
- 그것과 함께 화이트 와인 한 잔 주실 수 있나요?
- 감사합니다, 지금은 이게 다입니다.

In [20]:
# 이번에는 question 을 '미국에서 피자 주문'으로 설정하여 실행합니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "미국에서 피자 주문"})
# 스트리밍 출력
stream_response(answer)

- 영어 회화:
  A: Hi, I’d like to order a pizza for delivery, please.
  B: Sure, what would you like?
  A: I’d like a large pepperoni pizza with extra cheese and a side of garlic bread.
  B: Anything to drink?
  A: Yes, a 2-liter bottle of Coke, please.
  B: Great. Can I have your address?
  A: It’s 123 Maple Street.
  B: And your phone number?
  A: 555-1234.
  B: Thank you. Your total comes to $24.95. It should take about 30 minutes for delivery. Is there anything else I can help you with?
  A: No, that’s all. Thank you!

- 한글 해석:
  A: 안녕하세요, 배달로 피자를 주문하고 싶습니다.
  B: 네, 무엇을 드릴까요?
  A: 큰 사이즈의 페퍼로니 피자에 치즈를 추가하고, 마늘빵도 하나 주세요.
  B: 음료는 필요하신가요?
  A: 네, 콜라 2리터 한 병 주세요.
  B: 알겠습니다. 주소를 알려주시겠어요?
  A: 123 메이플 스트리트입니다.
  B: 전화번호는 어떻게 되나요?
  A: 555-1234입니다.
  B: 감사합니다. 총 금액은 $24.95이며, 배달은 대략 30분 정도 소요될 예정입니다. 더 도와드릴 것이 있으실까요?
  A: 아니요, 그게 다입니다. 감사합니다!